<a href="https://colab.research.google.com/github/FG2511/ARE/blob/master/uHooProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
@File name: uHooProject.ipynb
@Created on 2018-12-14
@Authors: Federica Gerina, Francesca Moi, Silvia Maria Massa
@Description: Given a time-series dataset that contains minute-by-minute data 
about different kind of gases, collected by the uHoo air quality sensor, train
a NN that classifies if a minute belongs to the class "Pasto" (1) otherwise to
the class "Other" (0).
'''
import numpy as np
from pandas import DataFrame
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.preprocessing import sequence
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

#FIRST STEP: PREPARING THE DATA
print("Preparing the data...")

#Loading dataset
dataframe = read_csv('uHooDataset.csv', usecols=["Date and time","Temperature","Relative Humidity","PM2.5","TVOC","CO2","CO","Air Pressure","Ozone","NO2","Activity"])

#Converting class 'Pasto' to 1 and class 'Other' to 0
encoder = LabelEncoder()
dataframe['Activity'] = encoder.fit_transform(['Other', 'Pasto'])

#Converting values to float32
dataset = dataframe.values
dataset = dataset.astype('float32')

#Normalizing and rescaling the data
mean = dataset[:].mean(axis=0)
dataset -= mean
std = dataset[:].std(axis=0)
dataset /= std

#Converting timeseries into supervised learning problem ???

#Determining data for training, validation and test set
tr_rate = 30
val_rate = 40
train_start = round(int(len(dataset) * (tr_rate / 100)))
validation_start = round(train_start + int(len(dataset)) * (val_rate / 100))
test_start = validation_row + 1

#Generator parameters
#lookback = 144
#steps = 96 
#delay = 96 
#batchSize = 64 

#Creating generators
#trainig_gen = generator(dataset,lookback=lookback, delay=delay, min_index=0, max_index=train_start, shuffle=True, batch_size=batchSize, step=steps)
#validation_gen = generator(dataset,lookback=lookback, delay=delay, min_index=train_start+1, max_index=validation_start, shuffle=True, batch_size=batchSize, step=steps)
#test_gen = generator(dataset, lookback=lookback, delay=delay, min_index=test_start, max_index=None, batch_size=batchSize, step=steps)

#SECOND STEP: TRAINING THE LSTM MODEL
print("Training the model...")

#Simple LSTM model
model = Sequential()
model.add(LSTM(128, activation='softmax', return_sequences = True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(256, activation='softmax', dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1)) #activation'sigmoid' ? 

model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])

#history = model.fit(input_train, y_train,epochs=3,batch_size=128, validation_split=0.2)

model.save('lstm_model.h5')


#THIRD STEP: MODEL EVALUATION AND PLOTTING RESULTS

#Evaluate model
print("Evaluate model...")

#Plot results
print("Plot results...")

In [0]:
'''
@Description: Transforms a timeseries into a supervised learning problem: 
              organization of data into input and output patterns where the 
              observation at the previous time step is used as an input to 
              forecast the observation at the current time timestep.
@Source code: https://machinelearningmastery.com/use-timesteps-lstm-networks-time-series-forecasting
'''
def timeseries_to_supervised(data, lag=1):
	df = DataFrame(data)
	columns = [df.shift(i) for i in range(1, lag+1)]
	columns.append(df)
	df = concat(columns, axis=1)
	return df

In [0]:
'''
@Description: Generator method for timestep forecasting.
@Source code: "Deep Learning with Python", Francois Chollet
'''
'''
@param: 
  - data: the original array of floating-point data (nomalized).
  - lookback: how many timesteps back the input data should go.
  - delay: how many timesteps in the future the target should be.
  - min_index and max_index: indices in the data array that delimit which timesteps
    to draw from. This is useful for keeping a segment of the data for validation
    and another for testing.
  - shuffle: whether to shuffle the samples or draw them in chronological order.
  - batch_size: the number of samples per batch.
  - step: the period, in timesteps, at which you sample data.
'''
def generator(data, lookback, delay, min_index, max_index, shuffle=False, batch_size, step):
  if max_index is None:
    max_index = len(data) - delay - 1
  i = min_index + lookback
  
  while 1:
    if shuffle:
      rows = np.random.randint(min_index + lookback, max_index, size=batch_size)
    else:
        if i + batch_size >= max_index:
          i = min_index + lookback
        rows = np.arange(i, min(i + batch_size, max_index))
        i += len(rows)
        
    samples = np.zeros((len(rows), lookback // step, data.shape[-1]))
    targets = np.zeros((len(rows),))
    
    for j, row in enumerate(rows):
      indices = range(rows[j] - lookback, rows[j], step)
      samples[j] = data[indices]
      targets[j] = data[rows[j] + delay][1]
    yield samples, targets